 # Genetic Algorithm Tip Recommendor
 Ayra Qutub

 ---
This report discusses the design and implementation of a genetic algorithm system that recommends a tip percentage based on a multi-dimensional analysis of food quality and service quality. The project builds on the "Tipper" example in the skfuzzy library and extends it by implementing a genetic tree structure, using the EasyGA library, allowing the program to evaluate individual attributes that collectively define food and service quality. These are then used to recommend a tip amount.

In [ ]:
!pip install EasyGA
!pip install scikit-fuzzy
import EasyGA
import random
import numpy as np
import os
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

## Defining Rules
The quality of the food is evaluated based on three inputs: temperature, flavor, and portion size. These are measured on a scale from 0 to 1, each representing how favorable the user determined these aspects to be, from poor to good.

Five fuzzy rules are defined to map combinations of these input qualities to an output food quality.

- Rule 1: If any attribute is rated as poor, then the food quality is poor.
- Rule 2: If all attributes are average, the food quality is average.
- Rule 3: If all attributes are good, the food quality is good.
- Rule 4: If any two attributes are good, the food quality is good
- Rule 5: If any attribute is average and no attributes are poor, the food quality is average

Service quality is calculated based on attentiveness, friendliness, and speed, again using a scale from 0 to 1.

Five fuzzy rules determine the service quality output:

- Rule 1: If any attribute is poor, the service quality is poor
- Rule 2: If all attributes are average, the service quality is average
- Rule 3: If all attributes are good, the service quality is good.
- Rule 4: If any two attributes are good, the service quality is good
- Rule 5: If any attribute is average and no attributes are poor, the service quality is average

The recommended tip percentage is determined based on the calculated values of food and service quality.

The function defines antecedents for food and service quality, each with three fuzzy sets (poor, average, and good). The consequent, tip, ranges from 0% to 30%, allowing for a graded scale of tips.

Six fuzzy rules govern the tip calculation:

- Rule 1: If both qualities are poor, the tip is low
- Rule 2: If both qualities are average, the tip is medium
- Rule 3: If both qualities are good, the tip is high
- Rule 4: If one quality is good and the other is average, the tip is high
- Rule 5: If one quality is good and the other is poor, the tip is medium
- Rule 6: If one quality is average and the other is poor, the tip is low

In [ ]:
def food_rules(temperature, flavor, portion_size, food_quality):
  rules = [
  ctrl.Rule(temperature['poor'] | flavor['poor'] | portion_size['poor'], food_quality['poor']),
  ctrl.Rule(temperature['average'] & flavor['average'] & portion_size['average'], food_quality['average']),
  ctrl.Rule(temperature['good'] & flavor['good'] & portion_size['good'], food_quality['good']),
  ctrl.Rule((temperature['good'] & flavor['good']) | (flavor['good'] & portion_size['good']) | (temperature['good'] & portion_size['good']), food_quality['good']),
  ctrl.Rule((temperature['average'] | flavor['average'] | portion_size['average']) & ~(temperature['poor'] | flavor['poor'] | portion_size['poor']), food_quality['average'])
  ]
  return rules

def service_rules(attentiveness, friendliness, speed, service_quality):
  rules = [
  ctrl.Rule(attentiveness['poor'] | friendliness['poor'] | speed['poor'], service_quality['poor']),
  ctrl.Rule(attentiveness['average'] & friendliness['average'] & speed['average'], service_quality['average']),
  ctrl.Rule(attentiveness['good'] & friendliness['good'] & speed['good'], service_quality['good']),
  ctrl.Rule((attentiveness['good'] & friendliness['good']) | (friendliness['good'] & speed['good']) | (attentiveness['good'] & speed['good']), service_quality['good']),
  ctrl.Rule((attentiveness['average'] | friendliness['average'] | speed['average']) & ~(attentiveness['poor'] | friendliness['poor'] | speed['poor']), service_quality['average'])
  ]
  return rules

def tip_rules(food, service, tip):
  rules = [
  ctrl.Rule(food['poor'] & service['poor'], tip['low']),
  ctrl.Rule(food['average'] & service['average'], tip['medium']),
  ctrl.Rule(food['good'] & service['good'], tip['high']),
  ctrl.Rule((food['good'] & service['average']) | (food['average'] & service['good']), tip['high']),
  ctrl.Rule((food['good'] & service['poor']) | (food['poor'] & service['good']), tip['medium']),
  ctrl.Rule((food['average'] & service['poor']) | (food['poor'] & service['average']), tip['low'])
  ]
  return rules


## Membership Functions
For the genetic system, we want the membership functions to be able to evolve so that we can determine which membership functions provide the best fit.

To achieve this, the membership functions for each variable is defined based on the associated chromosome values.

In [ ]:
def membership_functions(temperature, flavor, portion_size, food_quality, attentiveness, friendliness, speed, service_quality, food, service, tip):
  food_quality_ctrl = ctrl.ControlSystem(food_rules(temperature, flavor, portion_size, food_quality))
  food_sim = ctrl.ControlSystemSimulation(food_quality_ctrl)

  service_quality_ctrl = ctrl.ControlSystem(service_rules(attentiveness, friendliness, speed, service_quality))
  service_sim = ctrl.ControlSystemSimulation(service_quality_ctrl)

  tip_ctrl = ctrl.ControlSystem(tip_rules(food, service, tip))
  tip_sim = ctrl.ControlSystemSimulation(tip_ctrl)
  return food_sim, service_sim, tip_sim

## System Set Up
The fuzzy system is initialized by defining the antecedents and consequents for each controller.

### Food Quality:

Antecedents:
- Temperature
- Flavour
- Portion Size

Consequent:
- Food Quality

### Service Quality:

Antecedents:
- Attentiveness
- Friendliness
- Speed

Consequent:
- Service Quality

### Tip:

Antecedents:
- Food Quality
- Service Quality

Consequent:
- Tip

In [ ]:
def setup_fuzzy_system(chromosome):
  chromosome = [gene.value if hasattr(gene, 'value') else gene for gene in chromosome]
  chromosome = [item for sublist in chromosome for item in sublist]
  # print(chromosome[0:2])
  temp_pr = chromosome[0:2]
  temp_avg = chromosome[2:5]
  temp_gd = chromosome[5:7]
  flavor_pr = chromosome[7:9]
  flavor_avg = chromosome[9:12]
  flavor_gd = chromosome[12:14]
  portion_pr = chromosome[14:16]
  portion_avg = chromosome[16:19]
  portion_gd = chromosome[19:21]
  food_pr = chromosome[21:23]
  food_avg = chromosome[23:26]
  food_gd = chromosome[26:28]
  atten_pr = chromosome[28:30]
  atten_avg = chromosome[30:33]
  atten_gd = chromosome[33:35]
  frien_pr = chromosome[35:37]
  frien_avg = chromosome[37:40]
  frien_gd = chromosome[40:42]
  speed_pr = chromosome[42:44]
  speed_avg = chromosome[44:47]
  speed_gd = chromosome[47:49]
  service_pr = chromosome[49:51]
  service_avg = chromosome[51:54]
  service_gd = chromosome[54:56]
  tip_pr = chromosome[56:58]
  tip_avg = chromosome[58:61]
  tip_gd = chromosome[61:63]



  temperature = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'temperature')
  flavor = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'flavor')
  portion_size = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'portion_size')

  food_quality = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'food_quality')

  temperature['poor'] = fuzz.trimf(temperature.universe, [0] + temp_pr)
  temperature['average'] = fuzz.trimf(temperature.universe, temp_avg)
  temperature['good'] = fuzz.trimf(temperature.universe, temp_gd + [1])

  flavor['poor'] = fuzz.trimf(flavor.universe, [0] + flavor_pr)
  flavor['average'] = fuzz.trimf(flavor.universe, flavor_avg)
  flavor['good'] = fuzz.trimf(flavor.universe, flavor_gd + [1])

  portion_size['poor'] = fuzz.trimf(portion_size.universe, [0] + portion_pr)
  portion_size['average'] = fuzz.trimf(portion_size.universe, portion_avg)
  portion_size['good'] = fuzz.trimf(portion_size.universe, portion_gd + [1])

  food_quality['poor'] = fuzz.trimf(food_quality.universe, [0] + food_pr)
  food_quality['average'] = fuzz.trimf(food_quality.universe, food_avg)
  food_quality['good'] = fuzz.trimf(food_quality.universe, food_gd + [1])



  attentiveness = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'attentiveness')
  friendliness = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'friendliness')
  speed = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'speed')

  service_quality = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'service_quality')

  attentiveness['poor'] = fuzz.trimf(attentiveness.universe, [0] + atten_pr)
  attentiveness['average'] = fuzz.trimf(attentiveness.universe, atten_avg)
  attentiveness['good'] = fuzz.trimf(attentiveness.universe, atten_gd + [1])

  friendliness['poor'] = fuzz.trimf(friendliness.universe, [0] + frien_pr)
  friendliness['average'] = fuzz.trimf(friendliness.universe, frien_avg)
  friendliness['good'] = fuzz.trimf(friendliness.universe, frien_gd + [1])

  speed['poor'] = fuzz.trimf(speed.universe, [0] + speed_pr)
  speed['average'] = fuzz.trimf(speed.universe, speed_avg)
  speed['good'] = fuzz.trimf(speed.universe, speed_gd + [1])

  service_quality['poor'] = fuzz.trimf(service_quality.universe, [0] + service_pr)
  service_quality['average'] = fuzz.trimf(service_quality.universe, service_avg)
  service_quality['good'] = fuzz.trimf(service_quality.universe, service_gd + [1])
  # print(service_quality)



  food = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'food')
  service = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'service')

  tip = ctrl.Consequent(np.arange(0, 31, 1), 'tip')

  food['poor'] = fuzz.trimf(food.universe, [0] + food_pr)
  food['average'] = fuzz.trimf(food.universe, food_avg)
  food['good'] = fuzz.trimf(food.universe, food_gd + [1])

  service['poor'] = fuzz.trimf(service.universe, [0] + service_pr)
  service['average'] = fuzz.trimf(service.universe, service_avg)
  service['good'] = fuzz.trimf(service.universe, service_gd + [1])

  tip['low'] = fuzz.trimf(tip.universe, [0] + tip_pr)
  tip['medium'] = fuzz.trimf(tip.universe, tip_avg)
  tip['high'] = fuzz.trimf(tip.universe, tip_gd + [30])


  return temperature, flavor, portion_size, food_quality, attentiveness, friendliness, speed, service_quality, food, service, tip

## Executing Inference:
The consequent values are determined for the given inputs using the fuzzy logic system that we have set up.

In [ ]:
def execute_fuzzy_inference(food_sim, service_sim, tip_sim, inputs):
  food_sim.input['temperature'] = inputs['temperature']
  food_sim.input['flavor'] = inputs['flavor']
  food_sim.input['portion_size'] = inputs['portion_size']
  food_sim.compute()
  if 'food_quality' in food_sim.output:
      food_quality_out = food_sim.output['food_quality']
  else:
      food_quality_out = 0

  service_sim.input['attentiveness'] = inputs['attentiveness']
  service_sim.input['friendliness'] = inputs['friendliness']
  service_sim.input['speed'] = inputs['speed']
  service_sim.compute()
  if 'service_quality' in service_sim.output:
      service_quality_out = service_sim.output['service_quality']
  else:
      service_quality_out = 0

  tip_sim.input['food'] = food_quality_out
  tip_sim.input['service'] = service_quality_out
  tip_sim.compute()
  if 'tip' in tip_sim.output:
      predicted_tip = tip_sim.output['tip']
  else:
      predicted_tip = 0
  return predicted_tip

## Genetic Evolution
A chromosome is initialized with random values to be fed into the system. A fitness function is defined which minimizes error by comparing the system's output with the expected output. The system evolves to find the values which best fit the problem.

In [ ]:
import os
if os.path.exists("database.db"):
    os.remove("database.db")

ga = EasyGA.GA()
train_data = pd.read_csv('tipper_train.csv')
test_data = pd.read_csv('tipper_test.csv')
def initialize_chromosome():
  chromosome_data = [
      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 1) for _ in range(2)]),
      sorted([random.uniform(0, 1) for _ in range(3)]),
      sorted([random.uniform(0, 1) for _ in range(2)]),

      sorted([random.uniform(0, 30) for _ in range(2)]),
      sorted([random.uniform(0, 30) for _ in range(3)]),
      sorted([random.uniform(0, 30) for _ in range(2)])

  ]

  # print(chromosome_data)
  return chromosome_data
  # return [item for sublist in chromosome_data for item in sublist]


def fitness(chromosome):
  # print(chromosome)
  # print(chromosome[1])
  temperature, flavor, portion_size, food_quality, attentiveness, friendliness, speed, service_quality, food, service, tip = setup_fuzzy_system(chromosome)
  total_error = 0
  for index, row in train_data.iterrows():
    inputs = {
        'temperature': max(0, min(1, row['food temperature'])),
        'flavor': max(0, min(1, row['food flavor'])),
        'portion_size': max(0, min(1, row['portion size'])),
        'attentiveness': max(0, min(1, row['attentiveness'])),
        'friendliness': max(0, min(1, row['friendliness'])),
        'speed': max(0, min(1, row['speed of service']))
    }
    actual_tip = row['tip']
    food_sim, service_sim, tip_sim = membership_functions(temperature, flavor, portion_size, food_quality, attentiveness, friendliness, speed, service_quality, food, service, tip)
    predicted_tip = execute_fuzzy_inference(food_sim, service_sim, tip_sim, inputs)
    error = abs(actual_tip - predicted_tip) ** 2
    total_error += error
  return total_error/ len(train_data)

ga.fitness_function_impl = fitness
ga.chromosome_length = 27
ga.population_size = 5
ga.generation_goal = 10
ga.chromosome_impl = initialize_chromosome
ga.target_fitness_type = 'min'
ga.evolve()
print("genetic system")
ga.print_best_chromosome()

def initialize_original():
  original_chromosome = [
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [0.17, 0.33], [0.33, 0.5, 0.67], [0.67, 0.84],
      [5, 10], [10, 15, 20], [20, 25]
  ]
  return original_chromosome

oga= EasyGA.GA()
oga.fitness_function_impl = fitness
oga.chromosome_length = 27
oga.population_size = 1
oga.generation_goal = 1
oga.chromosome_impl = initialize_original
oga.target_fitness_type = 'min'
oga.evolve()
print("Original non-genetic system")
oga.print_best_chromosome()

genetic system
Best Chromosome 	: [[0.19014157479091354, 0.5664563503627388]][[0.2223779870976138, 0.6765344894074736, 0.8759016906404644]][[0.5941475624882727, 0.6584944516139141]][[0.5199244065304364, 0.902079314502572]][[0.12872809351431158, 0.27157841708568087, 0.6096645776611244]][[0.18980323039480596, 0.25067137637413783]][[0.2171765863559425, 0.3626466978337054]][[0.3912715656615082, 0.6232380775468314, 0.719031593860905]][[0.011255174835646198, 0.9760249485084707]][[0.5988020707368273, 0.7610350909476346]][[0.40328213191249296, 0.8279896338748832, 0.9573609342806672]][[0.24308781837187265, 0.5902747667621889]][[0.4306058314218699, 0.9607418834705604]][[0.6230061905372362, 0.6939684424011657, 0.9694819489277545]][[0.3202319162310384, 0.7993174190250777]][[0.36873745295163973, 0.9455620623785417]][[0.07688997232729655, 0.7352586679148392, 0.7779001265448092]][[0.5453682015669857, 0.9873337068510882]][[0.8204661678235152, 0.9927345132092531]][[0.3595720643127399, 0.444170094691207

I used the easyGA library to implement my previous non-genetic system as well for the sake of comparison. This system doesn't evolve since it is non-genetic so `oga.population_size = 1
oga.generation_goal = 1`.

The values in the 'chromosome' that is meant to represent the non-genetic system are the values derived from using automf() .

The fitness target is minimum so a lower fitness is better.

This means that, upon comparing the genetic to the non-genetic system, the genetic system outperformed the original.